Visit https://github.com/facebookresearch/colorlessgreenRNNs/tree/main/data for the Gulordova et al (2018) dataset

Visit https://github.com/TalLinzen/rnn_agreement for the Linzen et al (2016) dataset

In [ ]:
# Colab runtime type

# RAM resources
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime\n')
else:
  print('You are using a high-RAM runtime!\n')

# GPU resources
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Your runtime has 37.8 gigabytes of available RAM

You are using a high-RAM runtime!

Not connected to a GPU


In [ ]:
%%capture
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

%cd /gdrive/MyDrive/Colab Notebooks/ADL project

Mounted at /gdrive
/gdrive/MyDrive/Colab Notebooks/ADL project


In [ ]:
import pandas as pd

# Setting up verb singular-plural bank
We use the dataset from Gulordova et al, containing singular-plural pairs, to set up a bank of verb forms.

This bank is necessary for the setup of the control task.

In [ ]:
import pandas as pd

f = "./data/Gulordova/agreement/English/generated.tab"

usecols = [
           'pattern', 
           'constr_id', 
           'sent_id', 
           'correct_number', 
           'form', 
           'class', 
           'type', 
           'prefix', 
           'n_attr', 
           'punct', 
           'freq', 
           'len_context', 
           'len_prefix', 
           'sent']

df_gul = pd.read_csv(f, 
                  usecols=usecols,
                  index_col=None,
                  delimiter='\t',
                  )

# grouping by constr_id, sent_id and pattern so that we get correct-wrong pairs
df_gul = df_gul.set_index(['constr_id', 'sent_id', 'pattern'])
df_gul = df_gul.sort_index()

df_gul

correct_number     form    class  \
constr_id sent_id pattern                                                    
0         0       L__NOUN_VERB_VERB                 sing       is  correct   
                  L__NOUN_VERB_VERB                 sing      are    wrong   
                  R__VERB_NOUN_CCONJ_VERB           plur    visit  correct   
                  R__VERB_NOUN_CCONJ_VERB           plur   visits    wrong   
          1       L__NOUN_VERB_VERB                 sing   admits  correct   
...                                                  ...      ...      ...   
24        7       L__NOUN_VERB_VERB                 plur    comes    wrong   
          8       L__NOUN_VERB_VERB                 plur     want  correct   
                  L__NOUN_VERB_VERB                 plur    wants    wrong   
          9       L__NOUN_VERB_VERB                 plur   decide  correct   
                  L__NOUN_VERB_VERB                 plur  decides    wrong   

                                                type  \
constr_id sent_id pattern                              
0         0       L__NOUN_VERB_VERB         original   
                  L__NOUN_VERB_VERB         original   
                  R__VERB_NOUN_CCONJ_VERB   original   
                  R__VERB_NOUN_CCONJ_VERB   original   
          1       L__NOUN_VERB_VERB        generated   
...                                              ...   
24        7       L__NOUN_VERB_VERB        generated   
          8       L__NOUN_VERB_VERB        generated   
                  L__NOUN_VERB_VERB        generated   
          9       L__NOUN_VERB_VERB        generated   
                  L__NOUN_VERB_VERB        generated   

                                                                      prefix  \
constr_id sent_id pattern                                                      
0         0       L__NOUN_VERB_VERB                The thing to keep in mind   
                  L__NOUN_VERB_VERB                The thing to keep in mind   
                  R__VERB_NOUN_CCONJ_VERB          <unk> condemn the act but   
                  R__VERB_NOUN_CCONJ_VERB          <unk> condemn the act but   
          1       L__NOUN_VERB_VERB         The surplus to bother in version   
...                                                                      ...   
24        7       L__NOUN_VERB_VERB        Ladies who continue these experts   
          8       L__NOUN_VERB_VERB          Terms who require these actions   
                  L__NOUN_VERB_VERB          Terms who require these actions   
          9       L__NOUN_VERB_VERB         Mechanics who meet these updates   
                  L__NOUN_VERB_VERB         Mechanics who meet these updates   

                                           n_attr  punct    freq  len_context  \
constr_id sent_id pattern                                                       
0         0       L__NOUN_VERB_VERB             0  False  649202            5   
                  L__NOUN_VERB_VERB             0  False  261453            5   
                  R__VERB_NOUN_CCONJ_VERB       0  False    4859            4   
                  R__VERB_NOUN_CCONJ_VERB       0  False    1650            4   
          1       L__NOUN_VERB_VERB             0  False     637            5   
...                                           ...    ...     ...          ...   
24        7       L__NOUN_VERB_VERB             0  False    5252            5   
          8       L__NOUN_VERB_VERB             0  False    4591            5   
                  L__NOUN_VERB_VERB             0  False    1899            5   
          9       L__NOUN_VERB_VERB             0  False    1926            5   
                  L__NOUN_VERB_VERB             0  False    1624            5   

                                           len_prefix  \
constr_id sent_id pattern                               
0         0       L__NOUN_VERB_VERB                 6   
                  L__NOUN_VERB_V

In [ ]:
df_verbs = pd.DataFrame(columns=['sing', 'plur', 'freq_sing', 'freq_plur'])#, index=list(range(len(df_gul.index[::2]))))

# conditions
c1 = df_gul['correct_number'] == 'sing'
c2 = df_gul['correct_number'] == 'plur'

c3 = df_gul['class'] == 'correct'
c4 = df_gul['class'] == 'wrong'

# singular -> (singular & correct) | (plural & wrong)
df_verbs[['sing', 'freq_sing']] = df_gul.loc[(c1 & c3) | (c2 & c4)][['form', 'freq']]
# plural -> (plural & correct) | (singular & wrong)
df_verbs[['plur', 'freq_plur']] = df_gul.loc[(c2 & c3) | (c1 & c4)][['form', 'freq']]

df_verbs.drop_duplicates(ignore_index=True, inplace=True)

df_verbs

,sing,plur,freq_sing,freq_plur
0,is,are,649202,261453
1,visits,visit,1650,4859
2,admits,admit,637,722
3,refuses,refuse,1038,722
4,understands,understand,228,2723
...,...,...,...,...
92,fails,fail,1023,1242
93,reacts,react,232,561
94,suffers,suffer,493,1136
95,says,say,5308,5618


# Importing the Linzen et al (2016) dataset

In [ ]:
import pandas as pd

f = "./data/agr_50_mostcommon_10K.tsv"

usecols = ['sentence', 
           'orig_sentence', 
           'pos_sentence', 
           'subj', 
           'verb', 
        #    'subj_pos',
        #    'has_rel', 
        #    'has_nsubj', 
        #    'verb_pos', 
           'subj_index', 
           'verb_index',
           'n_intervening', 
        #    'last_intervening', 
        #    'n_diff_intervening', 
        #    'distance',
        #    'max_depth', 
        #    'all_nouns', 
        #    'nouns_up_to_verb'
           ]

df_lin = pd.read_csv(f,
                     index_col=None,
                     delimiter='\t',
                     usecols=usecols
                     )

df_lin.head()

,sentence,orig_sentence,pos_sentence,subj,verb,subj_index,verb_index,n_intervening
0,NNP and NNP VBD prints are not the issue .,a0 and a1 sized prints are not the issue .,NNP CC NNP VBD NNS VBP RB DT NN .,prints,are,5,6,0
1,a JJ NN ( or JJ or JJ NN ) is nothing but a JJ...,a 0-dimensional manifold ( or differentiable o...,DT JJ NN ( CC JJ CC JJ NN ) VBZ NN CC DT JJ JJ...,manifold,is,9,11,0
2,NNP NNP space ( NNP ) has some precedent in wo...,a0 non-breaking space ( nbsp ) has some preced...,NNP NNP NN ( NNP ) VBZ DT NN IN NN NNS VBN IN ...,space,has,3,7,0
3,"NNP , while the companion is magnitude 5 .","a0p , while the companion is magnitude 5 .","NNP , IN DT NN VBZ JJ CD .",companion,is,5,6,0
4,a JJ NN is not expected to get people VBG arou...,a 1000-items bibliography is not expected to g...,DT JJ NN VBZ RB VBN TO VB NNS VBG IN WP DT NN ...,author,does,14,16,0


# Filtering the dataset

1. Take subset with top-k frequency subject words
2. From subset take sentences with target verbs that are also contained in singular-plural verb bank
3. Sample 10k sentences from filtered selection

These steps are necessary to ensure comparability of the vocabulary with the control task.

In [ ]:
from tqdm import tqdm

# top-k frequency subject words
k=100
top_subj = df_lin['subj'].value_counts().index.tolist()[:k]

# empty dataframe to append to
df_lin_filt = pd.DataFrame(columns=df_lin.columns)

# create file
save_loc = './data/Linzen_filtered.csv'
df_lin_filt.to_csv(save_loc)

with tqdm(total= k * 2 * len(df_verbs)) as pbar:
    # for all top-k subjects
    for subj in tqdm(top_subj):
        df_subj = df_lin[df_lin['subj'] == subj]

        # for all verbs (singular and plural)
        for verb in df_verbs['sing'].tolist() + df_verbs['plur'].tolist():
            # save all rows containing both subj and verb
            df_subj[df_subj['verb'] == verb].to_csv(save_loc, header=False, mode='a')
            pbar.update(1)

In [ ]:
# in case filtering already executed, import set
save_loc = './data/Linzen_filtered_conditional.csv'
df_lin_filt = pd.read_csv(save_loc, index_col=0)
df_lin_filt.reset_index(drop=True, inplace=True)
df_lin_filt

,sentence,orig_sentence,pos_sentence,subj,verb,subj_index,verb_index,n_intervening
0,NNP did n't apply to this case as there was JJ...,a1 did n't apply to this case as there was amp...,NNP VBD RB VB IN DT NN IN EX VBD JJ NN TO VB W...,article,is,17,18,0
1,a 300 word article is a standard article .,a 300 word article is a standard article .,DT CD NN NN VBZ DT JJ NN .,article,is,4,5,0
2,"NNP ) for all NNS and purposes , the article i...","a3 ) for all intents and purposes , the articl...","NNP ) IN DT NNS CC NNS , DT NN VBZ JJ JJ NN .",article,is,10,11,0
3,"a7 , but the article is JJ for it 's tech NN .","a7 , but the article is incomprehensible for i...","NNP , CC DT NN VBZ JJ IN PRP VBZ JJ NN .",article,is,5,6,0
4,NNP ( and for those of you looking for a JJ re...,aaaa ( and for those of you looking for a non-...,"NNP ( CC IN DT IN PRP VBG IN DT JJ NN , IN DT ...",article,is,16,17,0
...,...,...,...,...,...,...,...,...
350828,"essentially , organs , JJ and nervous systems ...","essentially , organs , digestive and nervous s...","RB , NNS , JJ CC JJ NNS VBP IN NN IN NN CC RB ...",systems,suffer,8,9,0
350829,"in addition , NNP systems and water districts ...","in addition , lgu-managed systems and water di...","IN NN , NNP NNS CC NN NNS RB VBP IN PRP$ RB JJ...",systems,suffer,5,10,2
350830,that all these different number systems suffer...,that all these different number systems suffer...,IN PDT DT JJ NN NNS VBP IN JJ NNS IN DT JJ NNS...,systems,suffer,6,7,0
350831,"in NNP 's view , today 's american and , more ...","in wedel 's view , today 's american and , mor...","IN NNP POS NN , NN POS NNP CC , RBR RB , JJ JJ...",systems,look,21,22,0


In [ ]:
# sample subsets of rows to use as training/testing/validation dataset for finetuning
N = 5000

df_lin_subset = df_lin_filt.sample(N, 
                                   random_state=1,
                                   ignore_index=True)

df_lin_subset

,sentence,orig_sentence,pos_sentence,subj,verb,subj_index,verb_index,n_intervening
0,the problem with NNS ' is one of integrity and...,the problem with 'whiteshamans ' is one of int...,"DT NN IN NNS POS VBZ CD IN NN CC NN , RB IN NN...",problem,is,2,6,1
1,the characters also suffer from loss of identi...,the characters also suffer from loss of identi...,DT NNS RB VBP IN NN IN NN JJ IN DT NN .,characters,suffer,2,4,0
2,"the city of NNP is not part of the district , ...",the city of augsburg is not part of the distri...,"DT NN IN NNP VBZ RB NN IN DT NN , CC RB VBZ PR...",city,is,2,5,0
3,the main sources of JJ NNS are NNP .,the main sources of opaque inventories are hot...,DT JJ NNS IN JJ NNS VBP NNP .,sources,are,3,7,1
4,you missed the really big oversight though ; t...,you missed the really big oversight though ; t...,PRP VBD DT RB JJ NN IN : DT NN VBZ TO VB WRB D...,article,fails,10,11,0
...,...,...,...,...,...,...,...,...
4995,production company appears to be JJ .,production company appears to be non-existent .,NN NN VBZ TO VB JJ .,company,appears,2,3,0
4996,the name of the novel refers to the NNP charac...,the name of the novel refers to the kanji char...,DT NN IN DT NN VBZ IN DT NNP NNS IN DT NNP IN ...,name,refers,2,6,1
4997,the oldest part of the present church is in th...,the oldest part of the present church is in th...,DT JJS NN IN DT JJ NN VBZ IN DT NN CC NNS IN D...,part,is,3,8,1
4998,while people like NNP NNP and george abraham N...,while people like sampatti aryani and george a...,IN NNS IN NNP NNP CC NNP NNP NNP VBP IN NNP VB...,people,believe,2,10,0


In [ ]:
# reduce RAM usage
try:
    del df_gul
except:
    pass

try:
    del df_lin
except:
    pass

try:
    del df_lin_filt
except:
    pass

In [ ]:
# train/test/val split ratios: 0.8/0.1/0.1

from sklearn.model_selection import train_test_split

x_train, x_test_val = train_test_split(df_lin_subset, test_size=0.2, random_state=1)
x_test, x_val = train_test_split(x_test_val, test_size=0.5, random_state=1)

# Setting up BERT

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


# Tokenizing dataset

In [ ]:
import torch

# dataset class
class MaskedDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        # return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

from tqdm import tqdm

def get_tokenized(sentences, target_idxs, return_dataset=True):
    """
    Arguments:
    sentences - list of sentences to redact/encode
    target_idxs - indexes of targets that should be masked 
    return_dataset - default:True - return MaskedDataset class instance

    Returns: 
    encoded dataset with labels key
    
    Tokenization and encoding comparable.
    """

    # split sentences into lists of words
    sentences = [s.split(' ') for s in sentences]
    
    # redact target
    redacted = []
    for s, idx in tqdm(zip(sentences, target_idxs)):
        t = s.copy() # copy so that we don't alter the original
        t[idx - 1] = '[MASK]' # indexing starts at 1 in dataset
        redacted.append(t)
    
    # for arguments check docs: https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/tokenizer#transformers.PreTrainedTokenizer.__call__
    # tokenize redacted sentence
    encoded_redacted = tokenizer(redacted, is_split_into_words=True, return_tensors="pt", padding="max_length", truncation=True)
    # tokenize whole sentence for ground truth
    encoded_sentence = tokenizer(sentences, is_split_into_words=True, return_tensors="pt", padding="max_length", truncation=True)
    
    # didn't work, not entirely necessary anyway
        # we only need loss for the masked tokens, set all other tokens in the label to -100 so they are ignored
        # labels = encoded_sentence['input_ids'].detach().clone()
        # for i, (redact, label) in tqdm(enumerate(zip(encoded_redacted['input_ids'], labels))):
        #     mask_idx = redact.tolist().index(103) # get mask index in encoded sentence
        #     label_token = label[mask_idx] # get token value of unmasked label
            
        #     # labels[i] = torch.tensor([-100 for _ in range(len(label))]) # create tensor of only -100 tokens
        #     # labels[i][mask_idx] = label_token # replace label token with actual token id
            
        #     for j, token in enumerate(label):
        #         if j != mask_idx:
        #             label[i] = torch.tensor(-100)

    # attach encodings of unmasked sentences as labels
    # encoded_redacted['labels'] = labels
    encoded_redacted['labels'] = encoded_sentence['input_ids'].detach().clone()

    if return_dataset: return MaskedDataset(encoded_redacted)
    return encoded_redacted

In [ ]:
# tokenize data splits
X_train = get_tokenized(x_train['orig_sentence'],
                        x_train['verb_index'])

X_test = get_tokenized(x_test['orig_sentence'],
                       x_test['verb_index'],
                       return_dataset=False)

X_val = get_tokenized(x_val['orig_sentence'],
                      x_val['verb_index'])

4000it [00:00, 609415.76it/s]
500it [00:00, 693273.39it/s]
500it [00:00, 727672.45it/s]


# Finetuning BERT on downstream task

In [ ]:
# fine-tuning training parameters taken from Devlin et al (2019) (appendix A3)

from transformers import TrainingArguments

import datetime

time = datetime.datetime.now()
time_string = f"{time.year}-{time.month}-{time.day} {time.hour}.{time.minute}.{time.second}"

output_dir = "./out/runs/" + time_string + ' filt_dataset_3_epochs'

args = TrainingArguments(
    output_dir = output_dir,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    learning_rate = 5e-5,   # default
    num_train_epochs = 3,
    evaluation_strategy = "epoch"
)

from transformers import Trainer
# activate training mode
model.train()

trainer = Trainer(
    model = model,
    args = args,
    train_dataset = X_train,
    eval_dataset = X_val
)

trainer.train()

trainer.save_model(output_dir=output_dir)

# auto disconnect runtime after training + saving

from google.colab import runtime
runtime.unassign()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 750
  Number of trainable parameters = 109514298


Epoch,Training Loss,Validation Loss
1,No log,0.001286
2,0.250300,0.001211
3,0.250300,0.001204


***** Running Evaluation *****
  Num examples = 500
  Batch size = 16
Saving model checkpoint to ./out/runs/2022-12-17 14.58.28 filt_dataset_3_epochs/checkpoint-500
Configuration saved in ./out/runs/2022-12-17 14.58.28 filt_dataset_3_epochs/checkpoint-500/config.json
Model weights saved in ./out/runs/2022-12-17 14.58.28 filt_dataset_3_epochs/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 500
  Batch size = 16
***** Running Evaluation *****
  Num examples = 500
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in ./out/runs/2022-12-17 14.58.28 filt_dataset_3_epochs/config.json
Model weights saved in ./out/runs/2022-12-17 14.58.28 filt_dataset_3_epochs/pytorch_model.bin


# Constructing the control task dataset

1. Assign singular/plural labels randomly to unique subject word in dataset
2. Use singular/plural verb bank to assign new verb form to each row

In [ ]:
# make sure df_lin_filt is generated or imported first
df_control = df_lin_filt[['orig_sentence', 'subj', 'verb', 'subj_index', 'verb_index']]

# all available subject words
subjects = pd.Series(['not assigned']*len(df_control['subj'].unique()), index=df_control['subj'].unique())

# randomly assign singular or plural to each subject word
from numpy.random import seed, choice
seed(1)
subjects[:] = choice(['sing']*int(len(subjects)/2) + ['plur']*int(len(subjects)/2), size=len(subjects), replace=False)

# empty dataframe to append to in saved file
df_save = pd.DataFrame(columns=df_control.columns.tolist() + ['verb_new'])

# create file
save_loc = './data/Control.csv'
df_save.to_csv(save_loc)

# assign new verb labels to rows
from tqdm import tqdm
with tqdm(total= len(subjects) * 2 * len(df_verbs)) as pbar:
    for subj, number in zip(subjects.index, subjects.tolist()):
        # all rows with this subject word
        df_subj = df_control[df_control['subj'] == subj]
        
        # for all singular verbs
        for verb in df_verbs['sing'].tolist():
            # all rows containing both this subj and verb
            df_subj_verb = df_subj[df_subj['verb'] == verb].copy() # copy so we don't alter the original df

            # if empty dataframe turns up, skip
            if df_subj_verb.isnull().values.all():
                pbar.update(1)
                continue

            # assign new verb form
            df_subj_verb.loc[:, 'verb_new'] = df_verbs.loc[df_verbs['sing'] == verb, number].values[0]

            # append to save file
            df_subj_verb.to_csv(save_loc, header=False, mode='a')
            pbar.update(1)

        # for all plural verbs
        for verb in df_verbs['plur'].tolist():
            # all rows containing both this subj and verb
            df_subj_verb = df_subj[df_subj['verb'] == verb].copy() # copy so we don't alter the original df
            
            # if empty dataframe turns up, skip
            if df_subj_verb.isnull().values.all():
                pbar.update(1)
                continue
            
            # assign new verb form
            df_subj_verb.loc[:, 'verb_new'] = df_verbs.loc[df_verbs['plur'] == verb, number].values[0]

            # append to save file
            df_subj_verb.to_csv(save_loc, header=False, mode='a')
            pbar.update(1)

In [ ]:
# in case filtering already executed, import set
save_loc = './data/Control.csv'
df_control = pd.read_csv(save_loc, index_col=0)
df_control.reset_index(drop=True, inplace=True)
df_control

,orig_sentence,subj,verb,subj_index,verb_index,verb_new
0,a1 did n't apply to this case as there was amp...,article,is,17,18,are
1,a 300 word article is a standard article .,article,is,4,5,are
2,"a3 ) for all intents and purposes , the articl...",article,is,10,11,are
3,"a7 , but the article is incomprehensible for i...",article,is,5,6,are
4,aaaa ( and for those of you looking for a non-...,article,is,16,17,are
...,...,...,...,...,...,...
350828,"essentially , organs , digestive and nervous s...",systems,suffer,8,9,suffers
350829,"in addition , lgu-managed systems and water di...",systems,suffer,5,10,suffers
350830,that all these different number systems suffer...,systems,suffer,6,7,suffers
350831,"in wedel 's view , today 's american and , mor...",systems,look,21,22,looks


In [ ]:
# sample subsets of rows to use as training/testing/validation dataset for finetuning
N = 5000

df_control_subset = df_control.sample(N, 
                                      random_state=1,
                                      ignore_index=True)

df_control_subset

,orig_sentence,subj,verb,subj_index,verb_index,verb_new,replaced_sentence
0,the problem with 'whiteshamans ' is one of int...,problem,is,2,6,are,
1,the characters also suffer from loss of identi...,characters,suffer,2,4,suffers,
2,the city of augsburg is not part of the distri...,city,is,2,5,are,
3,the main sources of opaque inventories are hot...,sources,are,3,7,are,
4,you missed the really big oversight though ; t...,article,fails,10,11,fail,
...,...,...,...,...,...,...,...
4995,production company appears to be non-existent .,company,appears,2,3,appears,
4996,the name of the novel refers to the kanji char...,name,refers,2,6,refers,
4997,the oldest part of the present church is in th...,part,is,3,8,are,
4998,while people like sampatti aryani and george a...,people,believe,2,10,believe,


In [ ]:
# reduce RAM usage
try:
    del df_control
except:
    pass

In [ ]:
# replace target verb in original sentence with assigned verb form
df_new = df_control_subset.copy()
df_new['replaced_sentence'] = ''

from tqdm import tqdm
for i, row in tqdm(df_control_subset.iterrows(), total=len(df_control_subset)):
    # split sentence, replace word at target (verb) index, put sentence back together
    s = row['orig_sentence'].split(' ')
    s[row['verb_index'] - 1] = row['verb_new']
    df_new.loc[i, 'replaced_sentence'] = ' '.join(s)

df_control_subset = df_new

100%|██████████| 5000/5000 [00:01<00:00, 2705.24it/s]


In [ ]:
# train/test/val split ratios: 0.8/0.1/0.1

from sklearn.model_selection import train_test_split

x_train, x_test_val = train_test_split(df_control_subset, test_size=0.2, random_state=1)
x_test, x_val = train_test_split(x_test_val, test_size=0.5, random_state=1)

# Tokenizing control set

In [ ]:
# tokenize data splits
X_train = get_tokenized(x_train['replaced_sentence'],
                        x_train['verb_index'])

X_test = get_tokenized(x_test['replaced_sentence'],
                       x_test['verb_index'],
                       return_dataset=False)

X_val = get_tokenized(x_val['replaced_sentence'],
                      x_val['verb_index'])

4000it [00:00, 887965.28it/s]
500it [00:00, 553046.41it/s]
500it [00:00, 515777.67it/s]


# Setting up BERT for control task

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Finetuning BERT on control task

In [ ]:
# fine-tuning training parameters taken from Devlin et al (2019) (appendix A3)

from transformers import TrainingArguments

import datetime

time = datetime.datetime.now()
time_string = f"{time.year}-{time.month}-{time.day} {time.hour}.{time.minute}.{time.second}"

output_dir = "./out/runs/" + time_string + ' control_3_epochs'

args = TrainingArguments(
    output_dir = output_dir,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    learning_rate = 5e-5,   # default
    num_train_epochs = 3,
    evaluation_strategy = "epoch"
)

from transformers import Trainer
# activate training mode
model.train()

trainer = Trainer(
    model = model,
    args = args,
    train_dataset = X_train,
    eval_dataset = X_val
)

trainer.train()

trainer.save_model(output_dir=output_dir)

# auto disconnect runtime after training + saving

from google.colab import runtime
runtime.unassign()

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 750
  Number of trainable parameters = 109514298


Epoch,Training Loss,Validation Loss
1,No log,0.001892
2,0.251500,0.001481
3,0.251500,0.001450


***** Running Evaluation *****
  Num examples = 500
  Batch size = 16
Saving model checkpoint to ./out/runs/2022-12-17 20.38.9 control_3_epochs/checkpoint-500
Configuration saved in ./out/runs/2022-12-17 20.38.9 control_3_epochs/checkpoint-500/config.json
Model weights saved in ./out/runs/2022-12-17 20.38.9 control_3_epochs/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 500
  Batch size = 16
***** Running Evaluation *****
  Num examples = 500
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in ./out/runs/2022-12-17 20.38.9 control_3_epochs/config.json
Model weights saved in ./out/runs/2022-12-17 20.38.9 control_3_epochs/pytorch_model.bin
